# Adding more optimization parameters

In [ ]:
%load_ext autoreload
%autoreload 2

## What we will learn about
- Defining a MiniStudy containing a Josephson junction and multiple modes.
- Using parameters targeting nonlinearities.


## Now it is your time to extend the example!
### Goal: also optimize the qubit frequency, its anharmonicity and resonator-qubit chi.
### TODO
- Add a function called ``get_mini_study_resonator_qubit`` to the mini-studies file that isolates the qubit and readout resonator.
- Transform the inductance of the qubit's Josephson junction into a design variable.
- Add the targets for the new parameters to the corresponding file. Import and use ``param_nonlin`` to define targets for $\chi$ and the anharmonicity.
- Add a function called ``get_opt_targets_resonator_qubit`` to the optimization targets file. Use the targets predefined in the library.
- Add a function called ``get_plot_settings_resonator_qubit`` to the optimization targets file. Plot the qubit frequency in the same panel as the resonator frequency, while $\chi$ and the qubit anharmonicity should appear in a new panel. 


In [ ]:
# Library imports
import os
from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qdesignoptimizer.utils.utils import get_save_path
from qdesignoptimizer.utils.utils import close_ansys

# Project-specific imports
import names as n
import mini_studies as ms
import optimization_targets as ot
import parameter_targets as pt
import plot_settings as ps
from design import render_qiskit_metal_design, create_chip_and_gui

os.makedirs(os.path.dirname("out/"), exist_ok=True)
close_ansys()

In [ ]:
design, gui = create_chip_and_gui()
render_qiskit_metal_design(design, gui)

## Creating design analysis objects

In [ ]:
MINI_STUDY_GROUP = n.NBR_1
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)
MINI_STUDY = ms.get_mini_study_resonator_qubit(group=MINI_STUDY_GROUP)
opt_targets = ot.get_opt_target_resonator_qubit(MINI_STUDY_GROUP)
plot_settings = ps.get_plot_settings_resonator_qubit(MINI_STUDY_GROUP)

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)

design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path=get_save_path("out/", n.CHIP_NAME),
    update_design_variables=False,
    plot_settings=plot_settings,
)

## Optimization step

In [ ]:
group_runs = 6
group_passes = 10
for i in range(group_runs):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

## Results

### Eigenmodes and cross Kerr

In [ ]:
design_analysis.get_eigenmode_results()

In [ ]:
design_analysis.get_cross_kerr_matrix(iteration=-1)

### What you should see
![image info](./plots/frequencies_1.png)
![image info](./plots/nonlinearities_1.png)

## Update parameters and close Ansys

In [ ]:
design_analysis.overwrite_parameters()
close_ansys()